# Подгрузка пакетов и данных

In [1]:
cd ..

/Users/n.barsukov/WebstormProjects/ml-bug-detector


In [2]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm

from gensim.models.word2vec import Word2Vec

from shared.consts import FUNCTIONS_JSON_PROPERTIES_NAMES, PATH_TO_FUNCTIONS_STORAGE_JSON, PATH_TO_WORD2VEC_MODEL
from shared.helpers import (
    get_key_in_func_creation_dict, # получаем ключ из словаря/json c объявлениями функциями
    get_func_name_from_key # из ключа от словаря/json c объявлениями функциями извлекаем имя функции
)

Загрузим уже построенную модель Word2Vec

In [3]:
word2VecModel = Word2Vec.load(PATH_TO_WORD2VEC_MODEL)

Загрузим JSON, в котором мы сложили названия функций (и названия их арггументов)

In [4]:
functions_storage_dictionary = json.load(open(PATH_TO_FUNCTIONS_STORAGE_JSON))

# Отделение объявления функций от их использования

Функции с одним и тем же названием могут быть объявлены по-разному, поэтому объявления функций будут склдываться в словарь по принципу:

<div style="text-align: center; font-weight: bold;">Имя репозитория + имя функции</div>

Функции **get_key_in_func_creation_dict** и **get_func_name_from_key** помогут нам с этим правилом

In [5]:
def find_all_functions_creations(func_storage_dic):
    func_creation_dic = {}
    
    for filePath in func_storage_dic.keys():
        for function in func_storage_dic[filePath]:
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionDeclaration' or
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionExpression'
               ):
                func_name = function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
                func_creation_dict_key = get_key_in_func_creation_dict(func_name, filePath)
                if func_creation_dict_key in func_creation_dic:
                    pass
                    #print(func_creation_dict_key, 'уже есть в словаре')
                func_creation_dic[func_creation_dict_key] = {
                    f"{FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES}" : function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES],
                    f"{FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH}": filePath
                }
    
    return func_creation_dic

In [6]:
func_creation_dic = find_all_functions_creations(functions_storage_dictionary)

А теперь находим все ВЫЗОВЫ функций и смотрим, насколько их аргументы отличны объявления функции

In [7]:
def find_all_func_executions(func_storage_dic, func_creation_dic, onlyInDictionary = True):
    func_exec_storage = []
    
    for filePath in func_storage_dic.keys():
        for function in func_storage_dic[filePath]:
            func_name = function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
            func_creation_dict_key = get_key_in_func_creation_dict(func_name, filePath)
            
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'CallExpression' and # должна быть вызовом функции
                len(function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) > 0 and # должна иметь хоть один аргумент
                (func_creation_dict_key in func_creation_dic or not onlyInDictionary)# должна быть в нашем словаре
            ):
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH] = filePath
                func_exec_storage.append(function)
    
    return func_exec_storage

In [8]:
func_exec_storage = find_all_func_executions(functions_storage_dictionary, func_creation_dic)

In [9]:
print('Размер словаря с объявлениями функций: ', len(func_creation_dic))
print('Размер массива с вызовами функций: ', len(func_exec_storage))

Размер словаря с объявлениями функций:  158133
Размер массива с вызовами функций:  110682


Словарь с объявлениями функций - словарь, в котором ключ - название функции, а значение – массив с названиями переданных аргументов.

Пример: функция с названием **confirmDelete**

In [10]:
func_creation_dic[get_key_in_func_creation_dict('confirmDelete', 'scripts/Block8/PHPCI/public/assets/js/phpci.js')]

{'argumentsNames': ['url', 'subject', 'reloadAfter'],
 'filePath': 'scripts/Block8/PHPCI/public/assets/js/phpci.js'}

Массив с вызовами функций состоит из словарей следующего формата

In [11]:
list(filter(lambda func_exec: func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME] == 'sleep', func_exec_storage))

[{'type': 'CallExpression',
  'argumentsNames': ['millis'],
  'functionName': 'sleep',
  'filePath': 'scripts/replit/jsrepl/tools/closure-compiler/trunk/lib/rhino/src/mozilla/js/rhino/toolsrc/org/mozilla/javascript/tools/debugger/test.js'},
 {'type': 'CallExpression',
  'argumentsNames': [1000],
  'functionName': 'sleep',
  'filePath': 'scripts/replit/jsrepl/tools/closure-compiler/trunk/lib/rhino/src/mozilla/js/rhino/toolsrc/org/mozilla/javascript/tools/debugger/test.js'}]

# Класс для вызовов функций

In [12]:
from shared.classes import FunctionExecution

Импортированный класс определяет схожесть названий аргументов (в их векторной форме) в вызванной функции:

- c названиями аргументов (на тех же позициях) в момент объявления функции (массив с циферкам)
- векторную близость названий аргументов с названием функции (массив с циферкам)
- общую векторную близость с названиями аргументами в момент объявления функции (одна циферка)

Посмотрим, как это работает на примере

In [13]:
def find_func_exec_and_compare_with_func_creation(funcName):
    example_func_execs = list(
        filter(lambda func: func[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME].endswith(funcName), func_exec_storage)
    )
    for example_func_exec in example_func_execs:
        exec_func = FunctionExecution(example_func_exec, func_creation_dic, word2VecModel)

        print('Вызов функции:', exec_func.func_name, 'c аргументами', exec_func.args_names)
        print('Функция же была объявлена с словаре с аргументами:', exec_func.args_in_creation_dic)
        print(exec_func.args_similarity_with_creation_args)
        print('_________________________________________________')

In [14]:
find_func_exec_and_compare_with_func_creation('updateLine')
find_func_exec_and_compare_with_func_creation('windowSearch')

Вызов функции: updateLine c аргументами ['lastLine', '', 'lastSpans', 'estimateHeight']
Функция же была объявлена с словаре с аргументами: ['line', 'text', 'markedSpans', 'estimateHeight']
[0.40866297, None, -0.024005493, 1.0]
_________________________________________________
Вызов функции: windowSearch c аргументами ['strSelectorName', None, 900, 'strTop', 'strLeft', 'strWindowName', 'parameters', None]
Функция же была объявлена с словаре с аргументами: ['strPage', 'strHeight', 'strWidth', 'strTop', 'strLeft', 'strWindow', 'parameters', 'strValueID']
[0.4237818, None, None, 1.0, 1.0, None, 1.0, None]
_________________________________________________
Вызов функции: windowSearch c аргументами ['strSelectorName', 450, 650, 'strTop', 'strLeft', 'strWindowName', 'parameters', 'strValueID']
Функция же была объявлена с словаре с аргументами: ['strPage', 'strHeight', 'strWidth', 'strTop', 'strLeft', 'strWindow', 'parameters', 'strValueID']
[0.4237818, None, None, 1.0, 1.0, None, 1.0, 0.999999

# Реализация алгоритма по поиску сомнительных мест

Идея максимально проста. Проходим по массиву вызовов функции.

- По названию функции определяем в словаре с объявлениями функций, с какими названиями аргументов была объявлена эта функция
- Для каждой пары аргументов (название аргумента в момент вызовы функции и название аргумента в момент объявление функции) считаем близость их названий в векторной форме с помощью уже построенной модели word2Vec (это будет значение 0 до 1)
- Если эта схожесть меньше заданного порога (threshold), то выбрасываем предупреждение пользователю

### Threshold

In [15]:
MIN_SIMILARITY_THRESHOLD = 0.1

In [16]:
def if_any_arg_below_threshold(list_with_sims, min_sim_threshold = MIN_SIMILARITY_THRESHOLD):
    if not list_with_sims:
        return False
    
    is_below_threshold_args_similarities = list(
        map(lambda x: x and x < min_sim_threshold, list_with_sims))

    return any(is_below_threshold_args_similarities)

Порог не должен быть большой, потому что модели важно достичь высокого precision, а не recall.

Если наш алгоритм будет часто беспокоить пользователя по пустякам, предупреждая об ошибках там, где их нет (false positive), то нашим алгоритмом просто никто не будет пользоваться. Нужно беспокоить программиста только прям в самых вероятных случаях наличия ошибки. Ведь программисты люди ленивые и занятые... 

In [17]:
warnings_count = 0
for func_exec_item in func_exec_storage:
    func_exec = FunctionExecution(func_exec_item, func_creation_dic, word2VecModel)
    
    if len(func_exec.args_names) != len(func_exec.args_in_creation_dic):
        continue
    
    if_any_dissims_with_creation_arg = if_any_arg_below_threshold(func_exec.args_similarity_with_creation_args)
    if_any_dissims_with_func_name = if_any_arg_below_threshold(func_exec.args_similarity_with_func_name)
    if_total_args_sim_with_creation_args_low = func_exec.total_args_sim_with_creation_args < MIN_SIMILARITY_THRESHOLD
    
    if if_any_dissims_with_creation_arg and if_any_dissims_with_func_name and if_total_args_sim_with_creation_args_low:
        warnings_count += 1
        print('В файле', func_exec.func_path, 'проверь функцию', func_exec.func_name)
        print('Она вызвана с аргументами', func_exec.args_names)
        print('Но вот объявлена функция со следующими аргументами:')
        print(func_exec.args_in_creation_dic)
        print('Попарная схожесть аргументов (c названиями в объявленной функции):',func_exec.args_similarity_with_creation_args)
        print('Общая схожесть (c названиями в объявленной функции):', func_exec.total_args_sim_with_creation_args)
        print('Попарная схожесть аргументов c названием функции:', func_exec.args_similarity_with_func_name)
        print('_____________')

В файле scripts/01org/web-simulator/lib/ripple/ui/plugins/settings-dialog.js проверь функцию fill
Она вызвана с аргументами ['build', 'projectPath']
Но вот объявлена функция со следующими аргументами:
['action', 'prop']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.07789196, -0.021467714]
Общая схожесть (c названиями в объявленной функции): 0.08319147
Попарная схожесть аргументов c названием функции: [0.038251348, 0.03182174]
_____________
В файле scripts/01org/web-simulator/lib/ripple/ui/plugins/settings-dialog.js проверь функцию fill
Она вызвана с аргументами ['build', 'remoteInspector']
Но вот объявлена функция со следующими аргументами:
['action', 'prop']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.07789196, None]
Общая схожесть (c названиями в объявленной функции): 0.08061557
Попарная схожесть аргументов c названием функции: [0.038251348, None]
_____________
В файле scripts/01org/web-simulator/lib/ripple/xhr/jsonp.js проверь функци

В файле scripts/Dynalon/Rainy/Rainy/WebService/Admin/UI/built.js проверь функцию register
Она вызвана с аргументами ['filter', 'filterFilter']
Но вот объявлена функция со следующими аргументами:
['name', 'factory']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.13588534, 0.0844464]
Общая схожесть (c названиями в объявленной функции): 0.06553432
Попарная схожесть аргументов c названием функции: [0.019846605, 0.013888973]
_____________
В файле scripts/Dynalon/Rainy/Rainy/WebService/Admin/UI/built.js проверь функцию register
Она вызвана с аргументами ['uppercase', 'uppercaseFilter']
Но вот объявлена функция со следующими аргументами:
['name', 'factory']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.020518973, 0.14568694]
Общая схожесть (c названиями в объявленной функции): 0.09115514
Попарная схожесть аргументов c названием функции: [0.049638916, 0.13299632]
_____________
В файле scripts/Ecotrust/madrona/madrona/media/common/js/test/layout/pa

В файле scripts/OnsenUI/OnsenUI/build/js/onsenui_all.js проверь функцию service
Она вызвана с аргументами ['DeviceBackButtonHandler', '']
Но вот объявлена функция со следующими аргументами:
['name', 'constructor']
Попарная схожесть аргументов (c названиями в объявленной функции): [-0.049840547, None]
Общая схожесть (c названиями в объявленной функции): -0.011452924
Попарная схожесть аргументов c названием функции: [0.010583455, None]
_____________
В файле scripts/PaulKinlan/WebIntents/src/webintents.js проверь функцию addEventListener
Она вызвана с аргументами ['iframe', 'load', '', False]
Но вот объявлена функция со следующими аргументами:
['obj', 'type', 'func', 'capture']
Попарная схожесть аргументов (c названиями в объявленной функции): [None, 0.060533173, None, None]
Общая схожесть (c названиями в объявленной функции): 0.079741046
Попарная схожесть аргументов c названием функции: [0.08878515, 0.15837161, None]
_____________
В файле scripts/PaulKinlan/WebIntents/src/webintents.js п

В файле scripts/allskystar/ECUI/scripts/input-control.js проверь функцию build
Она вызвана с аргументами ['blur']
Но вот объявлена функция со следующими аргументами:
['name']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.045709554]
Общая схожесть (c названиями в объявленной функции): 0.045709554
Попарная схожесть аргументов c названием функции: [0.01745951]
_____________
В файле scripts/allskystar/ECUI/scripts/input-control.js проверь функцию build
Она вызвана с аргументами ['focus']
Но вот объявлена функция со следующими аргументами:
['name']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.086221114]
Общая схожесть (c названиями в объявленной функции): 0.086221114
Попарная схожесть аргументов c названием функции: [-0.036723837]
_____________
В файле scripts/andyherbert/ansiedit/tools/changefont.js проверь функцию addButton
Она вызвана с аргументами ['default', '{}']
Но вот объявлена функция со следующими аргументами:
['className', 'args']


В файле scripts/appcelerator/alloy/test/specs/alloy.js проверь функцию describe
Она вызвана с аргументами ['alloy', '']
Но вот объявлена функция со следующими аргументами:
['description', 'specDefinitions']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.07878258, None]
Общая схожесть (c названиями в объявленной функции): 0.03973865
Попарная схожесть аргументов c названием функции: [-0.008121745, None]
_____________
В файле scripts/appcelerator/titanium_mobile/demos/KitchenSink-Nook/Resources/examples/json.js проверь функцию testRow
Она вызвана с аргументами ['array', '[1, 2, 3]', '']
Но вот объявлена функция со следующими аргументами:
['name', 'str', 'testfn']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.023615485, None, None]
Общая схожесть (c названиями в объявленной функции): 0.07360643
Попарная схожесть аргументов c названием функции: [-0.10902935, None]
_____________
В файле scripts/appcelerator-training/tcad-code/Starting_code/Andro

...

В файле scripts/tutao/tutanota/web/test/lib/chaj-as-promised.js проверь функцию method
Она вызвана с аргументами ['rejectedWith', '']
Но вот объявлена функция со следующими аргументами:
['name', 'asserter']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.033810172, None]
Общая схожесть (c названиями в объявленной функции): 0.047825612
Попарная схожесть аргументов c названием функции: [0.07141167, None]
_____________
В файле scripts/tutao/tutanota/web/test/lib/chaj-as-promised.js проверь функцию method
Она вызвана с аргументами ['become', '']
Но вот объявлена функция со следующими аргументами:
['name', 'asserter']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.08329038, None]
Общая схожесть (c названиями в объявленной функции): 0.09957012
Попарная схожесть аргументов c названием функции: [0.0071818624, None]
_____________
В файле scripts/twilson63/ngUpload/libs/angular.js проверь функцию register
Она вызвана с аргументами ['filter', 'filterFi

В файле scripts/vitalets/lazy-model/test/angular-1.3.0-rc.5.js проверь функцию ensureSafeMemberName
Она вызвана с аргументами ['key3', 'fullExp']
Но вот объявлена функция со следующими аргументами:
['name', 'fullExpression']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.008909587, 0.057491552]
Общая схожесть (c названиями в объявленной функции): 0.05654966
Попарная схожесть аргументов c названием функции: [0.28562036, 0.0196988]
_____________
В файле scripts/vitalets/lazy-model/test/angular-1.3.0-rc.5.js проверь функцию register
Она вызвана с аргументами ['filter', 'filterFilter']
Но вот объявлена функция со следующими аргументами:
['name', 'factory']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.13588534, 0.0844464]
Общая схожесть (c названиями в объявленной функции): 0.06553432
Попарная схожесть аргументов c названием функции: [0.019846605, 0.013888973]
_____________
В файле scripts/vitalets/lazy-model/test/angular-1.3.0-rc.5.js проверь

В файле scripts/weblion/diazo_responsivetheme/plonetheme/diazo_responsivetheme/theme/js/accessibility.js проверь функцию createCookie
Она вызвана с аргументами ['fontsize', '$fontsize', 365]
Но вот объявлена функция со следующими аргументами:
['name', 'value', 'days']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.0018149014, None, None]
Общая схожесть (c названиями в объявленной функции): 0.05354769
Попарная схожесть аргументов c названием функции: [-0.020353606, 0.15009177, None]
_____________
В файле scripts/webrtcHacks/ChadWallaceHart.com/easyrtc/js/demo_audio_video.js проверь функцию disable
Она вызвана с аргументами ['hangupButton']
Но вот объявлена функция со следующими аргументами:
['domId']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.059596963]
Общая схожесть (c названиями в объявленной функции): 0.059596963
Попарная схожесть аргументов c названием функции: [-0.023353137]
_____________
В файле scripts/webrtcHacks/ChadWallaceHart

В файле scripts/yui/2in3/src/lib/yui/2.4.0/build/button/button.js проверь функцию setAttributeFromDOMAttribute
Она вызвана с аргументами ['type']
Но вот объявлена функция со следующими аргументами:
['p_sAttribute']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.008056298]
Общая схожесть (c названиями в объявленной функции): 0.008056298
Попарная схожесть аргументов c названием функции: [0.06280858]
_____________
В файле scripts/yui/2in3/src/lib/yui/2.4.0/build/button/button.js проверь функцию setAttributeFromDOMAttribute
Она вызвана с аргументами ['value']
Но вот объявлена функция со следующими аргументами:
['p_sAttribute']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.042045534]
Общая схожесть (c названиями в объявленной функции): 0.042045534
Попарная схожесть аргументов c названием функции: [0.00015431287]
_____________
В файле scripts/yui/2in3/src/lib/yui/2.4.0/build/button/button.js проверь функцию setAttributeFromDOMAttribute
Она вызва

В файле scripts/ziaukhan/html5-showcase/public/scripts/vendor/socket.io.js проверь функцию inherit
Она вызвана с аргументами ['HTMLFile', '']
Но вот объявлена функция со следующими аргументами:
['parent', 'extra']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.019107463, None]
Общая схожесть (c названиями в объявленной функции): 0.0009650454
Попарная схожесть аргументов c названием функции: [0.00262839, None]
_____________
В файле scripts/zuweie/ebtAdmin/Public/static/js/date.js проверь функцию add
Она вызвана с аргументами ['isLeapYear', '']
Но вот объявлена функция со следующими аргументами:
['name', 'method']
Попарная схожесть аргументов (c названиями в объявленной функции): [-0.05303971, None]
Общая схожесть (c названиями в объявленной функции): -0.00485026
Попарная схожесть аргументов c названием функции: [-0.073446535, None]
_____________
В файле scripts/zuweie/ebtAdmin/Public/static/js/date.js проверь функцию add
Она вызвана с аргументами ['getDayName', ''

In [18]:
print('Было зафиксировано', warnings_count, 'предупреждений')

Было зафиксировано 1374 предупреждений


Для контраста посмотрим кейсы, которые модель смогла определить как корректные

In [19]:
class RANGE_THRESHOLDS:
    MIN = 0.2
    MAX = 0.8

def if_all_args_in_range_thresholds(
    list_with_sims,
    min_threshold = RANGE_THRESHOLDS.MIN,
    max_threshold = RANGE_THRESHOLDS.MAX
):
    if not list_with_sims:
        return False
    
    is_above_threshold_args_similarities = list(
        map(lambda x: x and (x > min_threshold and x < max_threshold), list_with_sims)
    )

    return all(is_above_threshold_args_similarities)

In [20]:
for func_exec_item in func_exec_storage:
    func_exec = FunctionExecution(func_exec_item, func_creation_dic, word2VecModel)
    
    if len(func_exec.args_names) != len(func_exec.args_in_creation_dic):
        continue
    
    if_all_similar_with_creation = if_all_args_in_range_thresholds(func_exec.args_similarity_with_creation_args)
    if_all_similar_with_func_name = if_all_args_in_range_thresholds(func_exec.args_similarity_with_func_name)
    
    if if_all_similar_with_creation and if_all_similar_with_func_name:
        print('В файле', func_exec.func_path, 'функция', func_exec.func_name)
        print('Была вызвана с аргументами', func_exec.args_names)
        print('А объявлена функция со следующими аргументами:')
        print(func_exec.args_in_creation_dic)
        print('Попарная схожесть аргументов (c названиями в объявленной функции):',func_exec.args_similarity_with_creation_args)
        print('Общая схожесть (c названиями в объявленной функции):', func_exec.total_args_sim_with_creation_args)
        print('Попарная схожесть аргументов c названием функции:', func_exec.args_similarity_with_func_name)
        print('_____________')

В файле scripts/01org/web-simulator/lib/ripple/ui/plugins/settings-dialog.js функция disable
Была вызвана с аргументами ['launch']
А объявлена функция со следующими аргументами:
['action']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.20151126]
Общая схожесть (c названиями в объявленной функции): 0.20151126
Попарная схожесть аргументов c названием функции: [0.21143228]
_____________
В файле scripts/18F/fbopen/sample-www/bootstrap/js/html5shiv.js функция shivDocument
Была вызвана с аргументами ['document']
А объявлена функция со следующими аргументами:
['ownerDocument']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.34234777]
Общая схожесть (c названиями в объявленной функции): 0.34234777
Попарная схожесть аргументов c названием функции: [0.27554682]
_____________
В файле scripts/DataTables/DataTablesSrc/js/core/core.columns.js функция _fnCalculateColumnWidths
Была вызвана с аргументами ['settings']
А объявлена функция со следующими аргумен

В файле scripts/aptana/frameworks/plugins/com.yahoo.yui.2.6/samples/menu/assets/menuariaplugin.js функция setARIARole
Была вызвана с аргументами ['oMenuItemLabel', '_MENUITEM']
А объявлена функция со следующими аргументами:
['element', 'role']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.2568876, 0.20683597]
Общая схожесть (c названиями в объявленной функции): 0.3351213
Попарная схожесть аргументов c названием функции: [0.4719668, 0.34638375]
_____________
В файле scripts/aptana/frameworks/plugins/com.yahoo.yui.2.7/samples/carousel/assets/carouselariaplugin.js функция setARIARole
Была вызвана с аргументами ['oFirstChild', '_PRESENTATION']
А объявлена функция со следующими аргументами:
['element', 'role']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.22694664, 0.31382832]
Общая схожесть (c названиями в объявленной функции): 0.31844527
Попарная схожесть аргументов c названием функции: [0.551292, 0.69255054]
_____________
В файле scripts/aw

В файле scripts/cdnjs/cdnjs/ajax/libs/dojo/1.10.1/dojo.js.uncompressed.js функция promoteModuleToPlugin
Была вызвана с аргументами ['plugin']
А объявлена функция со следующими аргументами:
['pluginModule']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.21175389]
Общая схожесть (c названиями в объявленной функции): 0.21175389
Попарная схожесть аргументов c названием функции: [0.21408181]
_____________
В файле scripts/cdnjs/cdnjs/ajax/libs/dojo/1.10.1/dojo.js.uncompressed.js функция promoteModuleToPlugin
Была вызвана с аргументами ['plugin']
А объявлена функция со следующими аргументами:
['pluginModule']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.21175389]
Общая схожесть (c названиями в объявленной функции): 0.21175389
Попарная схожесть аргументов c названием функции: [0.21408181]
_____________
В файле scripts/cdnjs/cdnjs/ajax/libs/dojo/1.10.1/dojo.js.uncompressed.js функция promoteModuleToPlugin
Была вызвана с аргументами ['plugin']
А об

В файле scripts/dojo/dojo/dojo.js функция promoteModuleToPlugin
Была вызвана с аргументами ['plugin']
А объявлена функция со следующими аргументами:
['pluginModule']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.21175389]
Общая схожесть (c названиями в объявленной функции): 0.21175389
Попарная схожесть аргументов c названием функции: [0.21408181]
_____________
В файле scripts/dojo/dojo/dojo.js функция promoteModuleToPlugin
Была вызвана с аргументами ['plugin']
А объявлена функция со следующими аргументами:
['pluginModule']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.21175389]
Общая схожесть (c названиями в объявленной функции): 0.21175389
Попарная схожесть аргументов c названием функции: [0.21408181]
_____________
В файле scripts/dojo/dojo/dojo.js функция promoteModuleToPlugin
Была вызвана с аргументами ['plugin']
А объявлена функция со следующими аргументами:
['pluginModule']
Попарная схожесть аргументов (c названиями в объявленной фун

В файле scripts/financeCoding/dart-html5-animation/examples/ch04/grayscale/grayscale.dart.app.js функция $dynamicSetMetadata
Была вызвана с аргументами ['table']
А объявлена функция со следующими аргументами:
['inputTable']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.20443979]
Общая схожесть (c названиями в объявленной функции): 0.20443979
Попарная схожесть аргументов c названием функции: [0.32423943]
_____________
В файле scripts/financeCoding/dart-html5-animation/examples/ch04/loadimage/loadimage.dart.app.js функция $dynamicSetMetadata
Была вызвана с аргументами ['table']
А объявлена функция со следующими аргументами:
['inputTable']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.20443979]
Общая схожесть (c названиями в объявленной функции): 0.20443979
Попарная схожесть аргументов c названием функции: [0.32423943]
_____________
В файле scripts/financeCoding/dart-html5-animation/examples/ch04/spraypaint/spraypaint.dart.app.js функция $dy

...

В файле scripts/jsdelivr/jsdelivr/files/modernizr/2.8.3/modernizr.js функция shivDocument
Была вызвана с аргументами ['document']
А объявлена функция со следующими аргументами:
['ownerDocument']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.34234777]
Общая схожесть (c названиями в объявленной функции): 0.34234777
Попарная схожесть аргументов c названием функции: [0.27554682]
_____________
В файле scripts/jxcore/jxcore/deps/mozjs/src/jit-test/tests/arguments/strict-nested-eval.js функция eval
Была вызвана с аргументами ['code']
А объявлена функция со следующими аргументами:
['eval']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.24434645]
Общая схожесть (c названиями в объявленной функции): 0.24434645
Попарная схожесть аргументов c названием функции: [0.24434645]
_____________
В файле scripts/jxcore/jxcore/deps/mozjs/src/jit-test/tests/auto-regress/bug601393.js функция eval
Была вызвана с аргументами ['code']
А объявлена функция со следующи

В файле scripts/okfn/ckanjs-deprecated/widgets/datapreview/lib/slickgrid/slick.columnpicker.js функция setColumns
Была вызвана с аргументами ['visibleColumns']
А объявлена функция со следующими аргументами:
['columnDefinitions']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.26021078]
Общая схожесть (c названиями в объявленной функции): 0.26021078
Попарная схожесть аргументов c названием функции: [0.28328237]
_____________
В файле scripts/okfn/recline/vendor/slickgrid/2.0.1/slick.grid.js функция setSortColumns
Была вызвана с аргументами ['sortColumns']
А объявлена функция со следующими аргументами:
['cols']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.27239037]
Общая схожесть (c названиями в объявленной функции): 0.27239037
Попарная схожесть аргументов c названием функции: [0.5681232]
_____________
В файле scripts/onokumus/Bootstrap-Admin-Template/dist/assets/lib/wysihtml5x/dist/wysihtml5x.js функция insertAfter
Была вызвана с аргументами

В файле scripts/revisitors/glitcher/glitcher.js функция copy
Была вызвана с аргументами ['dupe']
А объявлена функция со следующими аргументами:
['rgba']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.22296473]
Общая схожесть (c названиями в объявленной функции): 0.22296473
Попарная схожесть аргументов c названием функции: [0.30658388]
_____________
В файле scripts/rstoyanchev/spring-websocket-portfolio/src/main/webapp/assets/lib/html5shiv/src/html5shiv.js функция shivDocument
Была вызвана с аргументами ['document']
А объявлена функция со следующими аргументами:
['ownerDocument']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.34234777]
Общая схожесть (c названиями в объявленной функции): 0.34234777
Попарная схожесть аргументов c названием функции: [0.27554682]
_____________
В файле scripts/saw/touch-interfaces/chapter09/listing9.18.js функция goTo
Была вызвана с аргументами ['currentSlide']
А объявлена функция со следующими аргументами:
['di

В файле scripts/thangchung/magazine-website-mvc-5/_oldprojects/planning-poker/Cik.PlanningPoker.WebSolution/Cik.PP.Web/Scripts/vendors/fullcalendar/fullcalendar.js функция reportEvents
Была вызвана с аргументами ['cache']
А объявлена функция со следующими аргументами:
['events']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.21018736]
Общая схожесть (c названиями в объявленной функции): 0.21018736
Попарная схожесть аргументов c названием функции: [0.2508256]
_____________
В файле scripts/thangchung/magazine-website-mvc-5/_oldprojects/planning-poker/Cik.PlanningPoker.WebSolution/Cik.PP.Web/Scripts/vendors/fullcalendar/fullcalendar.js функция reportEvents
Была вызвана с аргументами ['cache']
А объявлена функция со следующими аргументами:
['events']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.21018736]
Общая схожесть (c названиями в объявленной функции): 0.21018736
Попарная схожесть аргументов c названием функции: [0.2508256]
_____________


В файле scripts/twitter/plumage.js/dist/plumage.js функция setSortColumns
Была вызвана с аргументами ['sortColumns']
А объявлена функция со следующими аргументами:
['cols']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.27239037]
Общая схожесть (c названиями в объявленной функции): 0.27239037
Попарная схожесть аргументов c названием функции: [0.5681232]
_____________
В файле scripts/twitter/plumage.js/dist/plumage.js функция setColumns
Была вызвана с аргументами ['visibleColumns']
А объявлена функция со следующими аргументами:
['columnDefinitions']
Попарная схожесть аргументов (c названиями в объявленной функции): [0.26021078]
Общая схожесть (c названиями в объявленной функции): 0.26021078
Попарная схожесть аргументов c названием функции: [0.28328237]
_____________
В файле scripts/twostairs/paperwork/frontend/app/js/html5shiv.js функция shivDocument
Была вызвана с аргументами ['document']
А объявлена функция со следующими аргументами:
['ownerDocument']
Попарная с

## Эксперименты с моделями

### Трансформируем данные

Оставим только функции, у которых 2 аргумента

In [ ]:
func_creation_dic_2_args = dict(
    filter(lambda elem: len(elem[1][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) == 2, func_creation_dic.items())
)

In [ ]:
class TWO_ARGS_FUNC_COLUMNS:
    FIRST_ARG = 'arg1'
    SECOND_ARG = 'arg2'
    FUNC_NAME = 'funcName'
    TARGET = 'isBuggy'  # В верном ли порядке расставлены аргументы в функции

In [ ]:
two_args_funcs = pd.DataFrame.from_dict(func_creation_dic_2_args, orient='index')
two_args_funcs[[TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG]] = pd.DataFrame(
    two_args_funcs.argumentsNames.tolist(), index=two_args_funcs.index
)

two_args_funcs[TWO_ARGS_FUNC_COLUMNS.FUNC_NAME] = list(
    map(lambda rowName: get_func_name_from_key(rowName), two_args_funcs.index)
)
two_args_funcs.drop(['argumentsNames', 'filePath'], axis='columns', inplace=True)
two_args_funcs[TWO_ARGS_FUNC_COLUMNS.TARGET] = 0

two_args_funcs.reset_index(inplace=True, drop=True)

print('Количество строк:', len(two_args_funcs))
two_args_funcs.head(3)

Меняем местами аргументами и добавляем эти строчки как isBuggy = 1

(то есть получим 50% одного класса и 50% другого класса)

In [ ]:
buggy_code_rows = []

for rowIndex in tqdm(range(len(two_args_funcs))):
    new_row = {
        TWO_ARGS_FUNC_COLUMNS.FUNC_NAME: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME],
        TWO_ARGS_FUNC_COLUMNS.FIRST_ARG: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG],
        TWO_ARGS_FUNC_COLUMNS.SECOND_ARG: two_args_funcs.loc[rowIndex, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG],
        TWO_ARGS_FUNC_COLUMNS.TARGET: 1
    }
    buggy_code_rows.append(new_row)

two_args_funcs = two_args_funcs.append(pd.DataFrame(buggy_code_rows)).reset_index(drop=True)

In [ ]:
print('Количество строк:', len(two_args_funcs))
two_args_funcs.tail(3)

Теперь заменяем слова на их векторное представление из word2Vec.

Если например, каждое слово представлено в модели word2Vec как вектор из 300 значений, то название функции + 2 аргумента + 1 целевая перееменная (isBuggy) = 901 переменная в датасете

In [ ]:
def replace_string_column_with_vectors(pd_df, columnName, word2VecVocabulary = word2VecModel.wv):
    DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC = '0'
    
    vector_size = len(word2VecVocabulary[DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC])
    
    future_columns_names = []
    
    vectorized_df = pd.DataFrame()
    
    for i in range(vector_size):
        future_columns_names.append(columnName + str(i))
    
    for rowIndex in tqdm(range(len(pd_df))):
        word = pd_df.loc[rowIndex, columnName]
        
        if not checkIfWordInWord2VecDict(word, word2VecVocabulary):
            #word = DEFAULT_VALUE_IF_NOT_EXITS_IN_WORD2VEC
            newColumnsDf = pd.DataFrame(np.array([None] * vector_size).reshape(-1, vector_size), columns = future_columns_names)
        else:
            newColumnsDf = pd.DataFrame(
                word2VecVocabulary[word].reshape(-1, len(word2VecVocabulary[word])), columns = future_columns_names
            )
   
        vectorized_df = vectorized_df.append(newColumnsDf)
    
    return pd.concat([pd_df, vectorized_df.reset_index(drop=True)], axis=1)

In [ ]:
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, word2VecModel.wv)
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, word2VecModel.wv)
two_args_funcs = replace_string_column_with_vectors(two_args_funcs, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME, word2VecModel.wv)

two_args_funcs = two_args_funcs.drop(
    [TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME],
    axis=1
)

In [ ]:
two_args_funcs = two_args_funcs.dropna().reset_index(drop=True)

In [ ]:
len(two_args_funcs)

### Обучаем

Кормим это дело модели

In [ ]:
y = np.array(two_args_funcs[TWO_ARGS_FUNC_COLUMNS.TARGET])

X = two_args_funcs.drop(TWO_ARGS_FUNC_COLUMNS.TARGET, axis = 1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000, random_state = 27)

rf.fit(X, y);

### Проверяем на обучающей

In [ ]:
from sklearn import metrics

y_train_pred = rf.predict(X)
print('accuracy:', metrics.accuracy_score(y, y_train_pred))
print('recall:', metrics.recall_score(y, y_train_pred))
print('precision:', metrics.precision_score(y, y_train_pred))

### Проверяем на тестовой выборке

Формируем тестовую выборку из массива с вызовами функциями

Вновь фильтруем только функции с 2 аргументами

In [ ]:
func_exec_storage_all = find_all_func_executions(functions_storage_dictionary, func_creation_dic, False)

func_exec_storage_2_args = list(filter(lambda func: len(func[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) == 2, func_exec_storage_all))

превращаем в датафрейм

In [ ]:
func_exec_2_args_df = pd.DataFrame(func_exec_storage_2_args)

func_exec_2_args_df = func_exec_2_args_df.rename(
    columns={FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME: TWO_ARGS_FUNC_COLUMNS.FUNC_NAME}
)

func_exec_2_args_df[[TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG]] = pd.DataFrame(
    func_exec_2_args_df[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES].tolist(), index=func_exec_2_args_df.index
)

func_exec_2_args_df = func_exec_2_args_df.drop([
    FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE,
    FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES
], axis='columns')

In [ ]:
func_exec_2_args_df.head(3)

векторизуем перемененные 

In [ ]:
func_exec_2_args_df = replace_string_column_with_vectors(func_exec_2_args_df, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, word2VecModel.wv)
func_exec_2_args_df = replace_string_column_with_vectors(func_exec_2_args_df, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, word2VecModel.wv)
func_exec_2_args_df = replace_string_column_with_vectors(func_exec_2_args_df, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME, word2VecModel.wv)

func_exec_2_args_df = func_exec_2_args_df.dropna()

INFO_COLUMNS = [FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH, TWO_ARGS_FUNC_COLUMNS.FIRST_ARG, TWO_ARGS_FUNC_COLUMNS.SECOND_ARG, TWO_ARGS_FUNC_COLUMNS.FUNC_NAME]

test_2_args_exec_df = func_exec_2_args_df.drop(
    INFO_COLUMNS,
    axis='columns'
)

func_exec_2_args_df = func_exec_2_args_df[INFO_COLUMNS]

In [ ]:
func_exec_2_args_df.head(3)

In [ ]:
test_2_args_exec_df.head(3)

In [ ]:
PRED_PROB_COLUMN = TWO_ARGS_FUNC_COLUMNS.TARGET + '_pred'

func_exec_2_args_df[PRED_PROB_COLUMN] = list(map(lambda x: x[1], rf.predict_proba(test_2_args_exec_df) ))

Посмотрим, как распределены вероятности уверенности модели, что конкретный вызов функции - баг.

In [ ]:
import seaborn as sns
sns.set(color_codes=True, rc={'figure.figsize':(11.7,8.27)})

sns.distplot(func_exec_2_args_df[PRED_PROB_COLUMN]);

Нам важно, что было была высокая precision (пусть даже и при низком recall). То есть мы не хотим докучать разработчика лишними false positives (нашей моделью тогда просто никто не будет пользоваться)

In [ ]:
pd.set_option('display.max_rows', 500)

func_exec_2_args_df[func_exec_2_args_df[PRED_PROB_COLUMN] >= 0.7]

Теперь нам осталось пройти по предсказанным моделью как ошибочным вызовам функций и понять, а действительно ли они ошибочные